In [1]:
from __future__ import annotations

import cProfile
import pstats
from asyncio import sleep

In [2]:
from py_research.db import (
    DataSource,
    TableMap,
    SubTableMap,
    SubMap,
    SelIndex,
    DataBase
)

In [3]:
from schema import (
    Search,
    Project,
    Task,
    User,
    Membership,
    Organization,
    SearchResult,
    TestSchema,
)

In [4]:
from typing import Any


async def _dummy_loader(obj: dict[str, Any]) -> dict:
    await sleep(0.2)
    return {
        'name': obj["name"],
        'role': obj["role"],
        'age': obj["age"] + 1
    }

In [5]:
data_source = DataSource(
    target=Search,
    push={
        "resultCount": Search.result_count,
        "search": Search.term,
        "results": SubTableMap(
            target=Search.results,
            rel_map=TableMap(push={SearchResult.score}),
            push={
                "project_name": Project.name,
                "project_start": Project.start,
                "project_end": Project.end,
                "project_status": Project.status,
                "tasks": SubTableMap(
                    target=Project.tasks,
                    push={
                        "task_name": Task.name,
                        "task_assignees": SubTableMap(
                            target=Task.assignees,
                            push=User.name,
                            match_by=[User.name],
                        ),
                        "task_status": Task.status,
                    },
                ),
                "members": SubTableMap(
                    target=Project.members,
                    loader=_dummy_loader,
                    push={User.name, User.age},
                    match_by=[User.name],
                    rel_map=TableMap(
                        push={
                            Membership.role,
                        },
                    ),
                ),
            },
            pull={
                Project.org: SubMap(
                    push={
                        "organization_name": Organization.name,
                        "organization_address": Organization.address,
                        "organization_city": Organization.city,
                        "organization_countries": Organization.ccs,
                    },
                    match_by=[Organization.name, Organization.city],
                ),
                Project.number: SelIndex()
            },
        ),
    },
)

In [6]:
from datetime import date


data = {
    "resultCount": 3,
    "search": "test",
    "results": [
        {
            "score": 0.9,
            "project_name": "baking cake",
            "project_start": date(2020, 1, 1),
            "project_end": date(2020, 1, 4),
            "project_status": "done",
            "organization_name": "Bakery",
            "organization_address": "Main Street 1",
            "organization_city": "Bakerville",
            "organization_countries": ["USA", "Mexico", "Canada"],
            "tasks": [
                {
                    "task_name": "task1",
                    "task_assignees": ["John"],
                    "task_status": "todo",
                },
                {
                    "task_name": "task2",
                    "task_assignees": ["John"],
                    "task_status": "todo",
                },
                {
                    "task_name": "task3",
                    "task_assignees": ["Jane"],
                    "task_status": "done",
                },
            ],
            "members": [
                {"name": "John", "role": "baker", "age": 30},
                {"name": "John", "role": "manager", "age": 40},
            ],
        },
        {
            "score": 0.8,
            "project_name": "cleaning shoes",
            "project_start": date(2020, 1, 2),
            "project_end": date(2020, 1, 5),
            "project_status": "done",
            "organization_name": "Shoe Shop",
            "organization_address": "Main Street 2",
            "organization_city": "Shoetown",
            "organization_countries": ["USA", "Mexico"],
            "tasks": [
                {
                    "task_name": "task4",
                    "task_assignees": ["John"],
                    "task_status": "todo",
                },
                {
                    "task_name": "task5",
                    "task_assignees": ["Jane"],
                    "task_status": "todo",
                },
            ],
            "members": [
                {"name": "John", "role": "cleaner", "age": 25},
                {"name": "Jane", "role": "manager", "age": 35},
            ],
        },
        {
            "score": 0.7,
            "project_name": "fixing cars",
            "project_start": date(2020, 1, 3),
            "project_end": date(2020, 1, 6),
            "project_status": "started",
            "organization_name": "Car Shop",
            "organization_address": "Main Street 3",
            "organization_city": "Cartown",
            "organization_countries": ["USA", "Canada"],
            "tasks": [
                {
                    "task_name": "task6",
                    "task_assignees": ["John"],
                    "task_status": "todo",
                }
            ],
            "members": [
                {"name": "John", "role": "mechanic", "age": 45},
                {"name": "Jane", "role": "manager", "age": 55},
                {"name": "Jack", "role": "manager", "age": 65},
            ],
        },
    ],
}

In [7]:
from pathlib import Path
import warnings

with warnings.catch_warnings():
  warnings.simplefilter("error")
  with cProfile.Profile() as pr:
    db = DataBase(
      schema=TestSchema,
      # url=Path("./test.xlsx")
    )
    rec = await data_source.load([data], db=db)

    stats = pstats.Stats(pr)

In [8]:
# db[User].extract()

In [9]:
print(db[Project.members.x.tasks].select_str)
db[Project.members.x.tasks].df()

SELECT "schema.Project".number AS "schema.Project.number",
       "schema.Project.members"._id AS "schema.Project.members._id",
       "schema.Project.members.tasks"._id AS "schema.Project.members.tasks._id",
       "schema.Project.members.tasks"._id AS _id,
       "schema.Project.members.tasks".name AS name,
       "schema.Project.members.tasks".status AS status,
       "schema.Project.members.tasks".project_number AS project_number
FROM "schema_Project" AS "schema.Project"
JOIN "schema_Membership" AS "schema.Project.members._rel" ON "schema.Project.members._rel"._to_number = "schema.Project".number
JOIN "schema_User" AS "schema.Project.members" ON "schema.Project.members._rel"._from__id = "schema.Project.members"._id
JOIN "schema_Assignment" AS "schema.Project.members.tasks._rel" ON "schema.Project.members.tasks._rel"._from__id = "schema.Project.members"._id
JOIN "schema_Task" AS "schema.Project.members.tasks" ON "schema.Project.members.tasks._rel"._to__id = "schema.Project.members.t

schema.Project.number,schema.Project.members._id,_id,name,status,project_number
i64,str,str,str,str,i64
0,"""ad429663-47fd-408d-9db5-3db672…","""9444ec4a-9bb3-4f53-b9d3-9c1283…","""task4""","""todo""",1
0,"""ad429663-47fd-408d-9db5-3db672…","""9444ec4a-9bb3-4f53-b9d3-9c1283…","""task4""","""todo""",1
1,"""ad429663-47fd-408d-9db5-3db672…","""9444ec4a-9bb3-4f53-b9d3-9c1283…","""task4""","""todo""",1
1,"""ad429663-47fd-408d-9db5-3db672…","""9444ec4a-9bb3-4f53-b9d3-9c1283…","""task4""","""todo""",1
0,"""ad429663-47fd-408d-9db5-3db672…","""a313a3bd-6e70-4809-aad9-9ea2a6…","""task6""","""todo""",2
…,…,…,…,…,…
0,"""ad429663-47fd-408d-9db5-3db672…","""e90b9f91-7d29-41d4-be94-9604b7…","""task1""","""todo""",0
1,"""ad429663-47fd-408d-9db5-3db672…","""e90b9f91-7d29-41d4-be94-9604b7…","""task1""","""todo""",0
1,"""ad429663-47fd-408d-9db5-3db672…","""e90b9f91-7d29-41d4-be94-9604b7…","""task1""","""todo""",0


In [10]:
db.describe()


{
    'schema': {
        'package': 'py-research',
        'module': 'schema',
        'object': 'TestSchema',
        'package_version': '^5.1.1.dev152+g2c0bcb5.d20250123',
        'repo': 'https://github.com/cloudlane-one/py-research.git',
        'repo_revision': '1089c1ffb0ae83facec0f6a63ad964689e6caadd',
        'module_path': WindowsPath('exp/schema.py'),
        'module_dirty': False,
        'docs_url': 'https://cloudlane-one.github.io/py-research/'
    },
    'contents': {
        'records': {
            'schema.User': 3,
            'schema.Organization': 3,
            'schema.Project': 3,
            'schema.Search': 1,
            'schema.Task': 6
        },
        'arrays': {'schema.Organization_ccs': 7},
        'relations': {'schema.Assignment': 6, 'schema.SearchResult': 3, 'schema.Membership': 7}
    },
    'backend': None
}

In [11]:
nodes, edges = db.to_graph([Search, Project, Organization])

In [12]:
nodes

,node_id,schema.Search.term,schema.Search.result_count,table,schema.Project.number,schema.Project.name,schema.Project.start,schema.Project.end,schema.Project.status,schema.Project.org__id,schema.Organization._id,schema.Organization.name,schema.Organization.address,schema.Organization.city
0,0,test,3.0,schema_Search,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,NaN,NaN,schema_Project,0.0,baking cake,2020-01-01,2020-01-04,done,f23ca98c-7a20-4543-a90f-2ac8ee6d1c34,NaN,NaN,NaN,NaN
2,2,NaN,NaN,schema_Project,1.0,cleaning shoes,2020-01-02,2020-01-05,done,a6e6442a-ac5c-4981-87ef-4d2840b0a891,NaN,NaN,NaN,NaN
3,3,NaN,NaN,schema_Project,2.0,fixing cars,2020-01-03,2020-01-06,started,55cdf244-0d31-4eaf-aa48-0bb1d0a742f5,NaN,NaN,NaN,NaN
4,4,NaN,NaN,schema_Organization,NaN,NaN,NaN,NaN,NaN,NaN,a6e6442a-ac5c-4981-87ef-4d2840b0a891,Bakery,Main Street 1,Bakerville
5,5,NaN,NaN,schema_Organization,NaN,NaN,NaN,NaN,NaN,NaN,55cdf244-0d31-4eaf-aa48-0bb1d0a742f5,Shoe Shop,Main Street 2,Shoetown
6,6,NaN,NaN,schema_Organization,NaN,NaN,NaN,NaN,NaN,NaN,f23ca98c-7a20-4543-a90f-2ac8ee6d1c34,Car Shop,Main Street 3,Cartown


In [13]:
edges

,source,target,ltr,rtl,schema.SearchResult._id,schema.SearchResult.score,schema.SearchResult._from_term,schema.SearchResult._to_number
0,1,6,org__id,None,NaN,NaN,NaN,NaN
1,2,4,org__id,None,NaN,NaN,NaN,NaN
2,3,5,org__id,None,NaN,NaN,NaN,NaN
3,0,3,_to_number,_from_term,3467122831,0.7,test,2.0
4,0,1,_to_number,_from_term,4376575091,0.9,test,0.0
5,0,2,_to_number,_from_term,3823747280,0.8,test,1.0


In [ ]:
db[User].values()


[
    User({'_id': 'ad429663-47fd-408d-9db5-3db672d0d110', 'name': 'John', 'age': 31}),
    User({'_id': '4a68126c-589f-4c44-8189-da690ce06b5e', 'name': 'Jane', 'age': 56}),
    User({'_id': '9457beed-97a3-405d-a0b3-81251ee60111', 'name': 'Jack', 'age': 66})
]

In [15]:
db[Project].df()

number,name,start,end,status,org__id
i64,str,date,date,str,str
0,"""baking cake""",2020-01-01,2020-01-04,"""done""","""f23ca98c-7a20-4543-a90f-2ac8ee…"
1,"""cleaning shoes""",2020-01-02,2020-01-05,"""done""","""a6e6442a-ac5c-4981-87ef-4d2840…"
2,"""fixing cars""",2020-01-03,2020-01-06,"""started""","""55cdf244-0d31-4eaf-aa48-0bb1d0…"


In [ ]:
db[Search.term].values()

['test']

In [ ]:
list(db[Search.result_count].items())

[('test', 3)]

In [18]:
list(db[Search.results.x.org].items())


[
    (
        ('test', 2, '55cdf244-0d31-4eaf-aa48-0bb1d0a742f5'),
        Organization({'_id': '55cdf244-0d31-4eaf-aa48-0bb1d0a742f5', 'name': 'Shoe Shop', 'address': 'Main Street 2', 'city': 'Shoetown'})
    ),
    (
        ('test', 0, 'f23ca98c-7a20-4543-a90f-2ac8ee6d1c34'),
        Organization({'_id': 'f23ca98c-7a20-4543-a90f-2ac8ee6d1c34', 'name': 'Car Shop', 'address': 'Main Street 3', 'city': 'Cartown'})
    ),
    (
        ('test', 1, 'a6e6442a-ac5c-4981-87ef-4d2840b0a891'),
        Organization({'_id': 'a6e6442a-ac5c-4981-87ef-4d2840b0a891', 'name': 'Bakery', 'address': 'Main Street 1', 'city': 'Bakerville'})
    )
]

In [19]:
db[Search.results].items()


[
    (
        ('test', 2),
        Project({'number': 2, 'name': 'fixing cars', 'start': datetime.date(2020, 1, 3), 'end': datetime.date(2020, 1, 6), 'status': 'started'})
    ),
    (
        ('test', 0),
        Project({'number': 0, 'name': 'baking cake', 'start': datetime.date(2020, 1, 1), 'end': datetime.date(2020, 1, 4), 'status': 'done'})
    ),
    (
        ('test', 1),
        Project({'number': 1, 'name': 'cleaning shoes', 'start': datetime.date(2020, 1, 2), 'end': datetime.date(2020, 1, 5), 'status': 'done'})
    )
]

In [ ]:
list(db[Search.results]._abs_cols.keys())

['number', 'name', 'start', 'end', 'status', 'org__id']

In [ ]:
db[Search.results].df()

schema.Search.term,number,name,start,end,status,org__id
str,i64,str,date,date,str,str
"""test""",2,"""fixing cars""",2020-01-03,2020-01-06,"""started""","""55cdf244-0d31-4eaf-aa48-0bb1d0…"
"""test""",0,"""baking cake""",2020-01-01,2020-01-04,"""done""","""f23ca98c-7a20-4543-a90f-2ac8ee…"
"""test""",1,"""cleaning shoes""",2020-01-02,2020-01-05,"""done""","""a6e6442a-ac5c-4981-87ef-4d2840…"


In [ ]:
db[Search.results.x.org].df()

schema.Search.term,schema.Search.results.number,_id,name,address,city
str,i64,str,str,str,str
"""test""",2,"""55cdf244-0d31-4eaf-aa48-0bb1d0…","""Shoe Shop""","""Main Street 2""","""Shoetown"""
"""test""",0,"""f23ca98c-7a20-4543-a90f-2ac8ee…","""Car Shop""","""Main Street 3""","""Cartown"""
"""test""",1,"""a6e6442a-ac5c-4981-87ef-4d2840…","""Bakery""","""Main Street 1""","""Bakerville"""


In [ ]:
db[Organization].df()

_id,name,address,city
str,str,str,str
"""a6e6442a-ac5c-4981-87ef-4d2840…","""Bakery""","""Main Street 1""","""Bakerville"""
"""55cdf244-0d31-4eaf-aa48-0bb1d0…","""Shoe Shop""","""Main Street 2""","""Shoetown"""
"""f23ca98c-7a20-4543-a90f-2ac8ee…","""Car Shop""","""Main Street 3""","""Cartown"""


In [ ]:
db[SearchResult._table[SearchResult.score.isin(slice(0.81, 0.9))]].df()

_id,score,_from_term,_to_number
str,f64,str,i64
"""4376575091""",0.9,"""test""",0


In [ ]:
import pandas as pd


db[Organization.ccs].df(pd.DataFrame)

value
schema.Organization._id              schema.Organization.ccs.idx        
f23ca98c-7a20-4543-a90f-2ac8ee6d1c34 0                               USA
                                     1                            Mexico
                                     2                            Canada
a6e6442a-ac5c-4981-87ef-4d2840b0a891 0                               USA
                                     1                            Mexico
55cdf244-0d31-4eaf-aa48-0bb1d0a742f5 0                               USA
                                     1                            Canada

In [26]:
stats.sort_stats("cumulative")
stats.print_stats("py_research")

         478855 function calls (440534 primitive calls) in 0.855 seconds

   Ordered by: cumulative time
   List reduced from 2292 to 160 due to restriction <'py_research'>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
       55    0.000    0.000    0.310    0.006 C:\Users\lworm\Repos\py-research\src\py_research\db\datasources.py:697(_load_rec_from_item)
       55    0.002    0.000    0.306    0.006 C:\Users\lworm\Repos\py-research\src\py_research\db\datasources.py:572(_load_record)
       60    0.000    0.000    0.290    0.005 C:\Users\lworm\Repos\py-research\src\py_research\db\datasources.py:845(load)
   176/61    0.001    0.000    0.280    0.005 C:\Users\lworm\Repos\py-research\src\py_research\db\datasources.py:665(_load_records)
       19    0.000    0.000    0.255    0.013 C:\Users\lworm\Repos\py-research\src\py_research\db\databases.py:1522(__ior__)
       19    0.001    0.000    0.255    0.013 C:\Users\lworm\Repos\py-research\src\py_research\db\databas

      209    0.001    0.000    0.083    0.000 C:\Users\lworm\Repos\py-research\src\py_research\db\databases.py:698(__hash__)
 8013/243    0.008    0.000    0.065    0.000 C:\Users\lworm\Repos\py-research\src\py_research\hashing.py:54(inner)
 8013/243    0.017    0.000    0.064    0.000 C:\Users\lworm\Repos\py-research\src\py_research\hashing.py:69(gen_int_hash)
  948/269    0.001    0.000    0.062    0.000 C:\Users\lworm\Repos\py-research\src\py_research\hashing.py:43(_hash_sequence)
3424/2229    0.006    0.000    0.061    0.000 C:\Users\lworm\Repos\py-research\src\py_research\hashing.py:44(<lambda>)
       77    0.000    0.000    0.055    0.001 C:\Users\lworm\Repos\py-research\src\py_research\db\databases.py:2463(_sql_col)
    82/71    0.001    0.000    0.046    0.001 C:\Users\lworm\Repos\py-research\src\py_research\db\databases.py:2748(_gen_sql_base_table)
1951/1950    0.004    0.000    0.044    0.000 C:\Users\lworm\Repos\py-research\src\py_research\db\databases.py:2530(_get_typearg)